<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/MobileNetV2_initial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/Capstone dataset/Images-out'
train_dir = os.path.join(path, 'train')
validation_dir = os.path.join(path, 'val')



In [3]:
import tensorflow as tf
BATCH_SIZE = 32
IMG_SIZE = (100, 100)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 382 files belonging to 2 classes.


In [4]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 108 files belonging to 2 classes.


In [5]:
test_dir = os.path.join(path, 'test')

test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 57 files belonging to 2 classes.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [7]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [8]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [9]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [10]:
#Creating the base model
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [11]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 4, 4, 1280)


In [12]:
base_model.trainable = False
     

In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 50, 50, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 50, 50, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 50, 50, 32)   0           ['bn_Conv1[0][

In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [15]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [18]:
inputs = tf.keras.Input(shape=(100, 100, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [19]:
#Compiling the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 sequential (Sequential)     (None, None, None, 3)     0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 100, 100, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 100, 100, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 4, 4, 1280)       2257984   
 ional)                                                          
                                                           

In [21]:
len(model.trainable_variables)

2

In [22]:
initial_epochs = 15
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

Epoch 1/15
12/12 [==============================] - 60s 5s/step - loss: 0.7327 - accuracy: 0.5890 - val_loss: 0.6710 - val_accuracy: 0.6111
Epoch 2/15
12/12 [==============================] - 6s 390ms/step - loss: 0.7303 - accuracy: 0.5707 - val_loss: 0.6569 - val_accuracy: 0.6481
Epoch 3/15
12/12 [==============================] - 6s 363ms/step - loss: 0.7074 - accuracy: 0.6021 - val_loss: 0.6467 - val_accuracy: 0.6667
Epoch 4/15
12/12 [==============================] - 7s 364ms/step - loss: 0.6976 - accuracy: 0.5995 - val_loss: 0.6352 - val_accuracy: 0.6759
Epoch 5/15
12/12 [==============================] - 6s 373ms/step - loss: 0.7168 - accuracy: 0.6073 - val_loss: 0.6278 - val_accuracy: 0.6944
Epoch 6/15
12/12 [==============================] - 9s 582ms/step - loss: 0.6692 - accuracy: 0.6283 - val_loss: 0.6209 - val_accuracy: 0.6944
Epoch 7/15
12/12 [==============================] - 6s 363ms/step - loss: 0.6762 - accuracy: 0.6387 - val_loss: 0.6142 - val_accuracy: 0.7037
Epoch 8/

In [23]:
#Fine tuning the model
base_model.trainable = True

In [24]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [25]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 sequential (Sequential)     (None, None, None, 3)     0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 100, 100, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 100, 100, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 4, 4, 1280)       2257984   
 ional)                                                          
                                                           

In [27]:
len(model.trainable_variables)

56

In [28]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Epoch 15/25
12/12 [==============================] - 19s 898ms/step - loss: 0.6021 - accuracy: 0.6832 - val_loss: 0.5669 - val_accuracy: 0.7407
Epoch 16/25
12/12 [==============================] - 9s 663ms/step - loss: 0.5523 - accuracy: 0.7225 - val_loss: 0.5061 - val_accuracy: 0.7037
Epoch 17/25
12/12 [==============================] - 11s 682ms/step - loss: 0.5478 - accuracy: 0.7016 - val_loss: 0.6118 - val_accuracy: 0.7500
Epoch 18/25
12/12 [==============================] - 12s 916ms/step - loss: 0.5261 - accuracy: 0.7435 - val_loss: 0.5130 - val_accuracy: 0.7593
Epoch 19/25
12/12 [==============================] - 11s 821ms/step - loss: 0.5176 - accuracy: 0.7539 - val_loss: 0.6053 - val_accuracy: 0.7778
Epoch 20/25
12/12 [==============================] - 11s 822ms/step - loss: 0.4762 - accuracy: 0.7644 - val_loss: 0.5821 - val_accuracy: 0.7500
Epoch 21/25
12/12 [==============================] - 12s 775ms/step - loss: 0.5160 - accuracy: 0.7435 - val_loss: 0.5459 - val_accuracy: 

In [29]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

2/2 [==============================] - 15s 167ms/step - loss: 0.6744 - accuracy: 0.7895
Test accuracy : 0.7894737124443054
